In [1]:
import pandas as pd
import glob
import os
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def readDf():
    if not os.path.exists('./concat.csv'):
        path = 'C:/Projects/Bootcamp Ödev Cevapları/Untitled Folder/mart-eylül' # get path for all files that used by EDA
        all_files = glob.glob(path + "/*.txt")
        # create empty list and append all files
        file_list = []

        for filename in all_files:
            df = pd.read_csv(filename)
            file_list.append(df)

        # Concatenate all data into one DataFrame that called "mta"
        mta = pd.concat(file_list)
        mta.columns = mta.columns.str.strip()
        mta.to_csv('concat.csv', index=None)
        return mta
    else:
        return pd.read_csv('concat.csv')

In [3]:
mta = readDf()

In [4]:
mta['DATE_TIME'] = pd.to_datetime(mta["DATE"]+ " " + mta["TIME"], format='%m/%d/%Y %H:%M:%S')
mta['TURNSTILE'] = mta['C/A'] + ' ' + mta['UNIT'] + ' ' + mta['SCP'] + ' ' + mta['STATION']
mta.drop_duplicates(subset=['C/A', 'UNIT', 'SCP', 'STATION', 'DATE_TIME'], inplace=True)

In [5]:
mta.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,TURNSTILE
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/27/2021,03:00:00,REGULAR,7537255,2570434,2021-02-27 03:00:00,A002 R051 02-00-00 59 ST
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/27/2021,07:00:00,REGULAR,7537260,2570445,2021-02-27 07:00:00,A002 R051 02-00-00 59 ST
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/27/2021,11:00:00,REGULAR,7537285,2570500,2021-02-27 11:00:00,A002 R051 02-00-00 59 ST
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/27/2021,15:00:00,REGULAR,7537354,2570546,2021-02-27 15:00:00,A002 R051 02-00-00 59 ST
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/27/2021,19:00:00,REGULAR,7537501,2570582,2021-02-27 19:00:00,A002 R051 02-00-00 59 ST


In [6]:
mta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6279677 entries, 0 to 6279882
Data columns (total 13 columns):
 #   Column     Dtype         
---  ------     -----         
 0   C/A        object        
 1   UNIT       object        
 2   SCP        object        
 3   STATION    object        
 4   LINENAME   object        
 5   DIVISION   object        
 6   DATE       object        
 7   TIME       object        
 8   DESC       object        
 9   ENTRIES    int64         
 10  EXITS      int64         
 11  DATE_TIME  datetime64[ns]
 12  TURNSTILE  object        
dtypes: datetime64[ns](1), int64(2), object(10)
memory usage: 670.7+ MB


In [7]:
mta.shape

(6279677, 13)

In [8]:
mta.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS', 'DATE_TIME', 'TURNSTILE'],
      dtype='object')

In [9]:
#Checking null values based on columns

mta.isnull().sum()

C/A          0
UNIT         0
SCP          0
STATION      0
LINENAME     0
DIVISION     0
DATE         0
TIME         0
DESC         0
ENTRIES      0
EXITS        0
DATE_TIME    0
TURNSTILE    0
dtype: int64

In [10]:
mta.describe()

,ENTRIES,EXITS
count,6.279677e+06,6.279677e+06
mean,4.173093e+07,3.346020e+07
std,2.183711e+08,1.926419e+08
min,0.000000e+00,0.000000e+00
25%,2.188090e+05,9.744800e+04
50%,1.424042e+06,8.612790e+05
75%,6.052079e+06,3.990674e+06
max,2.147432e+09,2.133797e+09


In [11]:
mta['DESC'].value_counts()

REGULAR       6252548
RECOVR AUD      27129
Name: DESC, dtype: int64

In [12]:
##So we don't want the recovered audits. We want regular ones.
mta= pd.DataFrame(mta[mta.DESC=='REGULAR'])

In [13]:
mta['DESC'].value_counts()

REGULAR    6252548
Name: DESC, dtype: int64

In [14]:
mta['ENTRIES'] = mta['ENTRIES'].astype('int')
mta['EXITS'] = mta['EXITS'].astype('int')

In [15]:
mta['WEEKDAY'] = mta["DATE_TIME"].dt.day_name()

In [16]:
mta = mta[["STATION","TURNSTILE","DATE","TIME","DATE_TIME","WEEKDAY","ENTRIES","EXITS"]]
mta.sample(5)

,STATION,TURNSTILE,DATE,TIME,DATE_TIME,WEEKDAY,ENTRIES,EXITS
565315,145 ST,R178 R273 00-00-00 145 ST,03/17/2021,12:00:00,2021-03-17 12:00:00,Wednesday,4876343,5420855
4416489,25 ST,C017 R455 00-00-01 25 ST,07/27/2021,00:00:00,2021-07-27 00:00:00,Tuesday,1728184,2606534
3686361,SOUTH FERRY,R101 R001 02-00-01 SOUTH FERRY,06/27/2021,05:00:00,2021-06-27 05:00:00,Sunday,89004,66619
4507208,JAMAICA VAN WK,N604 R342 00-00-00 JAMAICA VAN WK,07/25/2021,01:00:00,2021-07-25 01:00:00,Sunday,6533240,12206449
3688395,WTC-CORTLANDT,R107D R305 04-03-00 WTC-CORTLANDT,06/30/2021,05:00:00,2021-06-30 05:00:00,Wednesday,1157754,84242


In [17]:
mta.sort_values(by='DATE_TIME')
mta['NO_OF_ENTRIES']= mta['ENTRIES'].diff()
mta['NO_OF_EXITS']= mta['EXITS'].diff()

In [18]:
mta.head(100)

,STATION,TURNSTILE,DATE,TIME,DATE_TIME,WEEKDAY,ENTRIES,EXITS,NO_OF_ENTRIES,NO_OF_EXITS
0,59 ST,A002 R051 02-00-00 59 ST,02/27/2021,03:00:00,2021-02-27 03:00:00,Saturday,7537255,2570434,NaN,NaN
1,59 ST,A002 R051 02-00-00 59 ST,02/27/2021,07:00:00,2021-02-27 07:00:00,Saturday,7537260,2570445,5.0,11.0
2,59 ST,A002 R051 02-00-00 59 ST,02/27/2021,11:00:00,2021-02-27 11:00:00,Saturday,7537285,2570500,25.0,55.0
3,59 ST,A002 R051 02-00-00 59 ST,02/27/2021,15:00:00,2021-02-27 15:00:00,Saturday,7537354,2570546,69.0,46.0
4,59 ST,A002 R051 02-00-00 59 ST,02/27/2021,19:00:00,2021-02-27 19:00:00,Saturday,7537501,2570582,147.0,36.0
...,...,...,...,...,...,...,...,...,...,...
95,59 ST,A002 R051 02-03-00 59 ST,02/28/2021,23:00:00,2021-02-28 23:00:00,Sunday,1416507,5346214,29.0,33.0
96,59 ST,A002 R051 02-03-00 59 ST,03/01/2021,03:00:00,2021-03-01 03:00:00,Monday,1416509,5346219,2.0,5.0
97,59 ST,A002 R051 02-03-00 59 ST,03/01/2021,07:00:00,2021-03-01 07:00:00,Monday,1416509,5346289,0.0,70.0
98,59 ST,A002 R051 02-03-00 59 ST,03/01/2021,11:00:00,2021-03-01 11:00:00,Monday,1416525,5346663,16.0,374.0


In [19]:
mta['TURNSTILE_TRAFFIC'] = mta['NO_OF_ENTRIES'] + mta['NO_OF_EXITS']
mta.describe()

,ENTRIES,EXITS,NO_OF_ENTRIES,NO_OF_EXITS,TURNSTILE_TRAFFIC
count,6.252548e+06,6.252548e+06,6.252547e+06,6.252547e+06,6.252547e+06
mean,4.172915e+07,3.346438e+07,-1.204581e+00,-4.109986e-01,-1.615580e+00
std,2.183567e+08,1.926481e+08,4.661486e+07,4.137038e+07,8.321259e+07
min,0.000000e+00,0.000000e+00,-2.144357e+09,-2.133741e+09,-3.960423e+09
25%,2.190480e+05,9.778200e+04,2.000000e+00,4.000000e+00,1.000000e+01
50%,1.425448e+06,8.624190e+05,2.400000e+01,2.600000e+01,6.500000e+01
75%,6.053489e+06,3.992673e+06,8.100000e+01,8.400000e+01,1.790000e+02
max,2.147432e+09,2.133797e+09,2.147040e+09,2.133741e+09,3.966472e+09


In [41]:
mta.sort_values(by='TURNSTILE_TRAFFIC').head()

,STATION,TURNSTILE,DATE,TIME,DATE_TIME,WEEKDAY,ENTRIES,EXITS,NO_OF_ENTRIES,NO_OF_EXITS,TURNSTILE_TRAFFIC
6168303,47-50 STS ROCK,N501 R020 01-03-03 47-50 STS ROCK,09/18/2021,00:00:00,2021-09-18,Saturday,2568505,1818054,-1.923261e+09,-2.037161e+09,-3.960423e+09
5958280,47-50 STS ROCK,N501 R020 01-03-03 47-50 STS ROCK,09/11/2021,00:00:00,2021-09-11,Saturday,2562258,1811792,-1.923260e+09,-2.037159e+09,-3.960419e+09
5748441,47-50 STS ROCK,N501 R020 01-03-03 47-50 STS ROCK,09/04/2021,00:00:00,2021-09-04,Saturday,2557289,1807226,-1.923257e+09,-2.037155e+09,-3.960412e+09
5538553,47-50 STS ROCK,N501 R020 01-03-03 47-50 STS ROCK,08/28/2021,00:00:00,2021-08-28,Saturday,2551923,1801745,-1.923257e+09,-2.037153e+09,-3.960410e+09
5330281,47-50 STS ROCK,N501 R020 01-03-03 47-50 STS ROCK,08/21/2021,00:00:00,2021-08-21,Saturday,2546351,1795953,-1.923255e+09,-2.037151e+09,-3.960406e+09


In [21]:
#Finding maximum and minimum date in dataframe

print("Maximum date: ")
print(mta["DATE_TIME"].max())
print()
print("Minimum date: ")
print(mta["DATE_TIME"].min())

Maximum date: 
2021-09-24 23:57:20

Minimum date: 
2021-02-27 00:00:00


In [23]:
mta.isnull().sum()

STATION              0
TURNSTILE            0
DATE                 0
TIME                 0
DATE_TIME            0
WEEKDAY              0
ENTRIES              0
EXITS                0
NO_OF_ENTRIES        1
NO_OF_EXITS          1
TURNSTILE_TRAFFIC    1
dtype: int64

In [33]:
mta = mta.dropna()
mta.isnull().sum()

STATION              0
TURNSTILE            0
DATE                 0
TIME                 0
DATE_TIME            0
WEEKDAY              0
ENTRIES              0
EXITS                0
NO_OF_ENTRIES        0
NO_OF_EXITS          0
TURNSTILE_TRAFFIC    0
dtype: int64

In [38]:
mta.groupby(['TURNSTILE','DATE']).sum().sort_values(ascending=False, by='TURNSTILE_TRAFFIC')

ENTRIES         EXITS  \
TURNSTILE                         DATE                                     
R523 R147 00-00-04 61 ST WOODSIDE 02/27/2021  1.106300e+10  1.273806e+10   
                                  03/06/2021  1.106298e+10  1.273801e+10   
                                  03/13/2021  1.106296e+10  1.273796e+10   
                                  03/20/2021  1.106294e+10  1.273791e+10   
                                  03/27/2021  1.106292e+10  1.273786e+10   
...                                                    ...           ...   
N501 R020 01-03-03 47-50 STS ROCK 08/21/2021  1.527853e+07  1.077615e+07   
                                  08/28/2021  1.531199e+07  1.081081e+07   
                                  09/04/2021  1.534414e+07  1.084365e+07   
                                  09/11/2021  1.537400e+07  1.087115e+07   
                                  09/18/2021  1.541144e+07  1.090881e+07   

                                              NO_OF_ENTRIES   NO_OF_EXITS  \
TURNSTILE                         DATE                                      
R523 R147 00-00-04 61 ST WOODSIDE 02/27/2021   1.843741e+09  2.122730e+09   
                                  03/06/2021   1.843735e+09  2.122715e+09   
                                  03/13/2021   1.843729e+09  2.122698e+09   
                                  03/20/2021   1.843723e+09  2.122681e+09   
                                  03/27/2021   1.843717e+09  2.122665e+09   
...                                                     ...           ...   
N501 R020 01-03-03 47-50 STS ROCK 08/21/2021  -1.923255e+09 -2.037151e+09   
                                  08/28/2021  -1.923256e+09 -2.037153e+09   
                                  09/04/2021  -1.923257e+09 -2.037155e+09   
                                  09/11/2021  -1.923260e+09 -2.037159e+09   
                                  09/18/2021  -1.923261e+09 -2.037161e+09   

                                              TURNSTILE_TRAFFIC  
TURNSTILE                         DATE                           
R523 R147 00-00-04 61 ST WOODSIDE 02/27/2021       3.966471e+09  
                                  03/06/2021       3.966450e+09  
                                  03/13/2021       3.966427e+09  
                                  03/20/2021       3.966405e+09  
                                  03/27/2021       3.966382e+09  
...                                                         ...  
N501 R020 01-03-03 47-50 STS ROCK 08/21/2021      -3.960406e+09  
                                  08/28/2021      -3.960409e+09  
                                  09/04/2021      -3.960412e+09  
                                  09/11/2021      -3.960419e+09  
                                  09/18/2021      -3.960422e+09  

[1048059 rows x 5 columns]